In [1]:
import os 

os.chdir("../..")

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "mps" if torch.backends.mps.is_available() else "cpu"

# === 4. Load and preprocess data ===
def load_datasets(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['label'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'label']]
    return Dataset.from_pandas(df)

train_dataset = load_datasets(paths.german_data_dir / "train_de.tsv")
val_dataset   = load_datasets(paths.german_data_dir / "dev_de.tsv")
test_dataset  = load_datasets(paths.german_data_dir / "dev_test_de.tsv")

W0614 14:53:55.252000 27100 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
model_name = "microsoft/deberta-v3-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/491 [00:00<?, ? examples/s]

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query_proj", "key_proj", "value_proj", "dense"] 
)

model = get_peft_model(model, lora_config).to(device)

In [6]:
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

In [ ]:
# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results/german-lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

# === 11. Evaluate on test set ===
trainer.evaluate(eval_dataset=test_dataset)

  0%|          | 0/2000 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.698, 'grad_norm': 1.0093352794647217, 'learning_rate': 4.975e-05, 'epoch': 0.05}
{'loss': 0.6847, 'grad_norm': 0.8912144899368286, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.1}
{'loss': 0.6779, 'grad_norm': 4.017972469329834, 'learning_rate': 4.9250000000000004e-05, 'epoch': 0.15}
{'loss': 0.6861, 'grad_norm': 2.537973165512085, 'learning_rate': 4.9e-05, 'epoch': 0.2}
{'loss': 0.6901, 'grad_norm': 1.059810996055603, 'learning_rate': 4.875e-05, 'epoch': 0.25}
{'loss': 0.6856, 'grad_norm': 1.1726713180541992, 'learning_rate': 4.85e-05, 'epoch': 0.3}
{'loss': 0.6725, 'grad_norm': 1.6457667350769043, 'learning_rate': 4.825e-05, 'epoch': 0.35}
{'loss': 0.6794, 'grad_norm': 1.1387596130371094, 'learning_rate': 4.8e-05, 'epoch': 0.4}
{'loss': 0.6957, 'grad_norm': 1.1148936748504639, 'learning_rate': 4.775e-05, 'epoch': 0.45}
{'loss': 0.675, 'grad_norm': 3.504206418991089, 'learning_rate': 4.75e-05, 'epoch': 0.5}
{'loss': 0.68, 'grad_norm': 1.607285499572754, 'learning_rate

  0%|          | 0/123 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


...

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 3138.4817, 'train_samples_per_second': 2.549, 'train_steps_per_second': 0.637, 'train_loss': 0.40653089388459923, 'epoch': 10.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 0.49777624011039734,
 'eval_accuracy': 0.8392857142857143,
 'eval_f1_macro': 0.8183783783783785,
 'eval_precision': 0.8126159554730983,
 'eval_recall': 0.8257387462027064,
 'eval_runtime': 15.5094,
 'eval_samples_per_second': 14.443,
 'eval_steps_per_second': 3.611,
 'epoch': 10.0}